# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [2]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

ModuleNotFoundError: No module named 'mplleaflet'

In [1]:
%matplotlib notebook
import datetime
from matplotlib import dates

df_source = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')

""" Define a few constants that help transforming stuff later on """
startDate = datetime.date(2005,1,1)
endDate = datetime.date(2014,12,31)
date2015 = datetime.date(2015,1,1)
date2016 = datetime.date(2016,1,1)


""" Get Data for first plot - record data for the years 2005 - 2014, highest and lowest for each day of the month """
df = df_source
df['Date'] = pd.to_datetime(df['Date'])
df = df[(df['Date'] >= startDate) & (df['Date'] <= endDate) & ~((df['Date'].dt.day == 29) & (df['Date'].dt.month == 2))]
df = df.sort_values(by=['Date', 'ID', 'Element'])
df = df.set_index('Date')
gdf = df.groupby([df.index.month, df.index.day])

""" Generate labels for the first plot"""
labels = list(gdf['Data_Value'].mean().index.map('{0[1]}/{0[0]}'.format))
labels = dates.date2num([datetime.datetime.strptime(d, "%d/%m").replace(year=2017) for d in labels])


""" Plot the min and max tempreatures for each day, filling in the space between """
plt.figure(figsize=(10, 6))
plt.plot(labels, (gdf['Data_Value'].max()/10).tolist(), labels, (gdf['Data_Value'].min()/10).tolist())
plt.gca().fill_between(labels
                       , (gdf['Data_Value'].max()/10).tolist()
                       , (gdf['Data_Value'].min()/10).tolist()
                       , facecolor='lightgrey'
                       , alpha=0.5
                       , label='Temperature differential 2005-2014')
hfmt = dates.DateFormatter('%b')
plt.gca().xaxis.set_major_formatter(hfmt)
plt.gca().xaxis.set_major_locator(dates.MonthLocator())


""" Get Data for the year 2015, combine it with the old data and find all occurences where the temperature in 2015 
is higher or lower than in the previous years """
df_2015 = df_source[(df_source['Date'] >= date2015) & (df_source['Date'] < date2016)]
df_2015 = df_2015.sort_values(by=['Date', 'ID', 'Element'])
df_2015 = df_2015.set_index('Date')
gdf_2015 = df_2015.groupby([df_2015.index.month, df_2015.index.day])

combinedValuesDf = pd.concat([gdf_2015['Data_Value'].max(), gdf_2015['Data_Value'].min(), gdf['Data_Value'].max(), gdf['Data_Value'].min()], axis = 1)
combinedValuesDf.columns = ['max2015', 'min2015', 'maxRecord', 'minRecord']
maxBrokenRecords = combinedValuesDf[combinedValuesDf['max2015'] > combinedValuesDf['maxRecord']].loc[:, ['max2015']]
minBrokenRecords = combinedValuesDf[combinedValuesDf['min2015'] < combinedValuesDf['minRecord']].loc[:, ['min2015']]

labelsMaxScatter = maxBrokenRecords.index.map('{0[1]}/{0[0]}'.format)
labelsMaxScatter = dates.date2num([datetime.datetime.strptime(d, "%d/%m").replace(year=2017) for d in labelsMaxScatter])

labelsMinScatter = minBrokenRecords.index.map('{0[1]}/{0[0]}'.format)
labelsMinScatter = dates.date2num([datetime.datetime.strptime(d, "%d/%m").replace(year=2017) for d in labelsMinScatter])

""" Generate two scatter plots where the temperature was higher/lower than in the prvious years """
plt.scatter(labelsMaxScatter, (maxBrokenRecords['max2015']/10).tolist(), c='red', label='Higher Temperatures in 2015')
plt.scatter(labelsMinScatter, (minBrokenRecords['min2015']/10).tolist(), c='green', label='Lower Temperatures in 2015')


""" Make the plot prettier """
xAxis = plt.gca().xaxis
yAxis = plt.gca().yaxis
plt.legend(loc=8 , frameon=False, title='Legend')
plt.title('Temperature Highs and Lows between 2005 and 2014')
for spine in plt.gca().spines.values():
    spine.set_visible(False)
    
plt.gca().set_ylabel('Temperature in C°')
    


NameError: name 'pd' is not defined